In [2]:
!pip install pandas

!pip install seaborn
!pip install numpy

!pip install matplotlib

!pip install scikit-learn

import matplotlib.pyplot as plt

import matplotlib

import seaborn as sns

import numpy as np

import pandas as pd

from sklearn import neighbors, metrics

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder

from collections import Counter

import re

from sklearn.covariance import EllipticEnvelope

from sklearn.linear_model import LinearRegression

from sklearn.naive_bayes import GaussianNB

from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import cross_val_score

You should consider upgrading via the 'd:\onedrive\pythonpr\samsung_knu_2021_practice\2021-knu-churn\venv\scripts\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'd:\onedrive\pythonpr\samsung_knu_2021_practice\2021-knu-churn\venv\scripts\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'd:\onedrive\pythonpr\samsung_knu_2021_practice\2021-knu-churn\venv\scripts\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'd:\onedrive\pythonpr\samsung_knu_2021_practice\2021-knu-churn\venv\scripts\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'd:\onedrive\pythonpr\samsung_knu_2021_practice\2021-knu-churn\venv\scripts\python.exe -m pip install --upgrade pip' command.


# functions

In [91]:
def get_metrics(targets: np.ndarray, predicted_targets: np.ndarray, mode: str):
    '''

    :param targets:
    :param predicted_targets:
    :param mode: influence to return: mode = "show_metrics" - print metrics to the Console;
                                        mode = "return_metrics" - return dict which consists all metrics values
                                        with their names as keys
    :return:
    {
    "accuracy": accuracy_value, "precision": precision_value,
    "recall": recall_value, "f1_score": f1_score_value,
    "confusion_matrix": confusion_matrix
    }
    '''
    # token = (actual target, predicted target)
    # 1 - positive target, 0 - negative target
    tokens = [(j, i) for i in [1, 0] for j in [1, 0]]
    # tokens = [true_pos_token,
    #           false_pos_token,
    #           false_neg_token,
    #           true_neg_token]
    c = Counter(zip(targets, predicted_targets))
    [true_pos_value,
     false_pos_value,
     false_neg_value,
     true_neg_value] = [c[token] for token in tokens]
    accuracy = (true_neg_value + true_pos_value) / len(targets)
    confusion_matrix = np.array([[true_pos_value, false_neg_value],
                                 [false_pos_value, true_neg_value]])
    if mode == "show_metrics":
        print(f"accuracy = {accuracy}")
        print(f"confusion_matrix = \n{confusion_matrix}")
    elif mode == "return_metrics":
        res_dict = {"accuracy": accuracy, "confusion_matrix": confusion_matrix}
        return res_dict

In [25]:
preproc_data = pd.read_csv("pre-processing_BankChurners.csv")
x = preproc_data.drop(["Attrition_Flag",
                       "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1",
                       "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2"
                       ], axis=1).values
y = preproc_data["Attrition_Flag"].values
preproc_data

,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
0,1,0.431818,1,3,2.0,2.0,70.000000,1,0.604651,5,...,0.340190,3.087008e-01,0.344983,0.499066,0.036260,0.250000,0.455055,0.061308,0.000087,0.999921
1,1,0.568182,1,3,4.0,2.0,100.000000,1,0.534884,4,...,0.059850,3.972984e-10,0.098765,0.969720,0.078753,0.078125,0.653319,0.000001,0.000014,0.999991
2,1,0.318182,0,4,2.0,2.0,26.666667,1,0.488372,3,...,0.056676,1.000000e+00,0.022779,0.525234,0.037804,0.078125,0.653319,0.763820,0.000127,0.999881
3,1,0.318182,1,3,1.0,2.0,70.000000,1,0.186047,5,...,0.099091,3.972984e-10,0.136382,0.813084,0.017501,0.140625,0.700084,0.000001,0.000015,0.999991
4,1,0.409091,1,2,4.0,2.0,50.000000,1,0.534884,3,...,0.077747,4.954311e-01,0.079783,0.514393,0.033057,0.109375,0.236909,0.312564,0.000048,0.999951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10115,1,0.545455,1,2,4.0,1.0,50.000000,1,0.627907,3,...,0.077536,7.353993e-01,0.062076,0.262804,0.855934,0.835938,0.239989,0.464323,0.000185,0.999821
10116,0,0.340909,1,2,4.0,3.0,50.000000,1,0.279070,4,...,0.085819,8.684942e-01,0.060308,0.300561,0.472062,0.460938,0.191263,0.513569,0.995689,0.004312
10117,0,0.409091,0,1,2.0,2.0,26.666667,1,0.534884,5,...,0.120042,3.972984e-10,0.156466,0.306169,0.559394,0.390625,0.229068,0.000001,0.998300,0.001700
10118,0,0.090909,1,2,4.0,2.0,50.000000,1,0.534884,4,...,0.116172,3.972984e-10,0.152756,0.200000,0.450958,0.406250,0.202185,0.000001,0.997130,0.002876


In [7]:
preproc_data.Attrition_Flag.value_counts()

1    8494
0    1626
Name: Attrition_Flag, dtype: int64

,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
0,0.431818,1,3,2.0,2.0,70.000000,1,0.604651,5,1,3,0.340190,3.087008e-01,0.344983,0.499066,0.036260,0.250000,0.455055,0.061308,0.999921
1,0.568182,1,3,4.0,2.0,100.000000,1,0.534884,4,1,0,0.059850,3.972984e-10,0.098765,0.969720,0.078753,0.078125,0.653319,0.000001,0.999991
2,0.318182,0,4,2.0,2.0,26.666667,1,0.488372,3,4,1,0.056676,1.000000e+00,0.022779,0.525234,0.037804,0.078125,0.653319,0.763820,0.999881
3,0.318182,1,3,1.0,2.0,70.000000,1,0.186047,5,1,0,0.099091,3.972984e-10,0.136382,0.813084,0.017501,0.140625,0.700084,0.000001,0.999991
4,0.409091,1,2,4.0,2.0,50.000000,1,0.534884,3,1,2,0.077747,4.954311e-01,0.079783,0.514393,0.033057,0.109375,0.236909,0.312564,0.999951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10115,0.545455,1,2,4.0,1.0,50.000000,1,0.627907,3,2,3,0.077536,7.353993e-01,0.062076,0.262804,0.855934,0.835938,0.239989,0.464323,0.999821
10116,0.340909,1,2,4.0,3.0,50.000000,1,0.279070,4,2,3,0.085819,8.684942e-01,0.060308,0.300561,0.472062,0.460938,0.191263,0.513569,0.004312
10117,0.409091,0,1,2.0,2.0,26.666667,1,0.534884,5,3,4,0.120042,3.972984e-10,0.156466,0.306169,0.559394,0.390625,0.229068,0.000001,0.001700
10118,0.090909,1,2,4.0,2.0,50.000000,1,0.534884,4,3,3,0.116172,3.972984e-10,0.152756,0.200000,0.450958,0.406250,0.202185,0.000001,0.002876


# decide what kind of model is suitable for your task
(supervised / unsupervised learning, classification / regression)
i chose supervised ml because we can organize train/test sets, with given dataset
but also we can try unsupervised ml and compare the results

# decide how to split data into train/test sets

In [118]:
from sklearn.ensemble import RandomForestClassifier

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.75)
print(f"shapes: \nx_train: {x_train.shape}, \nx_test: {x_test.shape}, \n"
      f"y_train: {y_train.shape}, \ny_test: {y_test.shape}")

max_test_size = 0.9
start_test_size = 0.6
test_size_step = 0.01
test_sizes = [round(start_test_size + i * test_size_step, 2)
              for i in list(range(int((max_test_size - start_test_size) / test_size_step) + 1))]
best_test_sizes = []
scores_dict = dict()
for rand_state_i in range(1, 5 + 1):
    for test_size_i in test_sizes:
        # print(f"test_size_i: {test_size_i}")
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size_i,
                                                            random_state=rand_state_i)
        clf_model = RandomForestClassifier(n_estimators=100)
        clf_model.fit(X=x_train, y=y_train)
        y_predict = clf_model.predict(x_test)
        y_pos_idx_array = [idx for idx, val in enumerate(y_test) if val == 1]
        y_neg_idx_array = [idx for idx, val in enumerate(y_test) if val == 0]

        metrics = get_metrics(targets=y_test[y_pos_idx_array], predicted_targets=y_predict[y_pos_idx_array], mode="return_metrics")
        pos_target_accuracy = metrics["accuracy"]

        metrics = get_metrics(targets=y_test[y_neg_idx_array], predicted_targets=y_predict[y_neg_idx_array], mode="return_metrics")
        neg_target_accuracy = metrics["accuracy"]

        scores_dict[test_size_i] = (pos_target_accuracy + neg_target_accuracy) / 2
        # print((pos_target_accuracy + neg_target_accuracy) / 2)
        best_test_size = max(scores_dict.items(), key=lambda x: x[1])[0]
        best_test_sizes.append(best_test_size)

best_test_size = sum(best_test_sizes) / len(best_test_sizes)
# best_test_size = max(scores_dict.items(), key=lambda x: x[1])[0]


shapes: 
x_train: (2530, 19), 
x_test: (7590, 19), 
y_train: (2530,), 
y_test: (7590,)
test_size_i: 0.6
test_size_i: 0.61
test_size_i: 0.62
test_size_i: 0.63
test_size_i: 0.64
test_size_i: 0.65
test_size_i: 0.66
test_size_i: 0.67
test_size_i: 0.68
test_size_i: 0.69
test_size_i: 0.7
test_size_i: 0.71
test_size_i: 0.72
test_size_i: 0.73
test_size_i: 0.74
test_size_i: 0.75
test_size_i: 0.76
test_size_i: 0.77
test_size_i: 0.78
test_size_i: 0.79
test_size_i: 0.8
test_size_i: 0.81
test_size_i: 0.82
test_size_i: 0.83
test_size_i: 0.84
test_size_i: 0.85
test_size_i: 0.86
test_size_i: 0.87
test_size_i: 0.88
test_size_i: 0.89
test_size_i: 0.9
test_size_i: 0.6
test_size_i: 0.61
test_size_i: 0.62
test_size_i: 0.63
test_size_i: 0.64
test_size_i: 0.65
test_size_i: 0.66
test_size_i: 0.67
test_size_i: 0.68
test_size_i: 0.69
test_size_i: 0.7
test_size_i: 0.71
test_size_i: 0.72
test_size_i: 0.73
test_size_i: 0.74
test_size_i: 0.75
test_size_i: 0.76
test_size_i: 0.77
test_size_i: 0.78
test_size_i: 0.79
t

In [119]:
best_test_size

0.6442580645161279

# choose the metrics to apply
accuracy, precision, recall, f1_score, confusion_matrix
according to our task we can understand that our goal is in success prediction of positive class of target feature as well as negative one,
so we have to test our model on sets with positive target feature and negative separately and the accuracy wil be the best choice for us
metrics to apply: accuracy

In [ ]:
y_pos_idx_array = [idx for idx, val in enumerate(y_test) if val == 1]
y_neg_idx_array = [idx for idx, val in enumerate(y_test) if val == 0]

# Naive Bayes model

In [52]:
gnb_model = GaussianNB()
gnb_model.fit(X=x_train, y=y_train)
y_predict = gnb_model.predict(X=x_test)

print("positive target prediction metrics")
get_metrics(y_test[y_pos_idx_array], y_predict[y_pos_idx_array], mode="show_metrics")
print("negative target prediction metrics")
get_metrics(y_test[y_neg_idx_array], y_predict[y_neg_idx_array], mode="show_metrics")
cross_val_score(gnb_model, x, y, cv=5, scoring="accuracy").mean()

positive target prediction metrics
accuracy = 0.9324260910370717
confusion_matrix = 
[[1987  144]
 [   0    0]]
negative target prediction metrics
accuracy = 0.6265664160401002
confusion_matrix = 
[[  0   0]
 [149 250]]


0.8281620553359685

# Random Forest

In [100]:
from sklearn.ensemble import RandomForestClassifier

clf_model = RandomForestClassifier(n_estimators=100)
clf_model.fit(X=x_train, y=y_train)
y_predict = clf_model.predict(x_test)
print("positive target prediction metrics")
get_metrics(y_test[y_pos_idx_array], y_predict[y_pos_idx_array], mode="show_metrics")
print("negative target prediction metrics")
get_metrics(y_test[y_neg_idx_array], y_predict[y_neg_idx_array], mode="show_metrics")
cross_val_score(clf_model, x, y, cv=5, scoring="accuracy").mean()

positive target prediction metrics
accuracy = 0.9568277803847959
confusion_matrix = 
[[1786   23]
 [  69  253]]
negative target prediction metrics
accuracy = 0.9523809523809523
confusion_matrix = 
[[317   2]
 [ 17  63]]


0.9263833992094863

# tune Random Forest algorithm parameters

In [97]:
from sklearn.model_selection import GridSearchCV
n_estimators = list(range(90, 140 + 10, 10))
max_features = ["auto", "sqrt"]
max_depth = [2, 4, 6, 8, 10 , 12]
bootstrap = [True, False]
param_grid = dict(n_estimators=n_estimators, max_features=max_features,
                  max_depth=max_depth, bootstrap=bootstrap)
grid_search = GridSearchCV(clf_model, param_grid=param_grid,
                           scoring="accuracy", cv=10)
grid_search.fit(X=x, y=y)
clf_model_best_params = grid_search.best_params_

In [98]:
clf_model_best_params

{'bootstrap': False,
 'max_depth': 8,
 'max_features': 'sqrt',
 'n_estimators': 80}

In [99]:
clf_model = RandomForestClassifier(**clf_model_best_params)
cross_val_score(clf_model, x, y, cv=5, scoring="accuracy").mean()

0.9152173913043476

# K-nearest neighbor algorithm

In [50]:
from sklearn.neighbors import KNeighborsClassifier

neigh_model = KNeighborsClassifier()
neigh_model.fit(X=x_train, y=y_train)
y_predict = neigh_model.predict(X=x_test)
print("positive target prediction metrics")
get_metrics(y_test[y_pos_idx_array], y_predict[y_pos_idx_array], mode="show_metrics")
print("negative target prediction metrics")
get_metrics(y_test[y_neg_idx_array], y_predict[y_neg_idx_array], mode="show_metrics")
cross_val_score(neigh_model, x, y, cv=5, scoring="accuracy").mean()

positive target prediction metrics
accuracy = 0.9751290473955889
confusion_matrix = 
[[2078   53]
 [   0    0]]
negative target prediction metrics
accuracy = 0.22556390977443608
confusion_matrix = 
[[  0   0]
 [309  90]]


0.8203557312252965

# tune K-nearest neighbor algorithm parameters

In [59]:
max_k = 30
knn_scores = []
for k in range(1, max_k + 1):
    neigh_model = KNeighborsClassifier(n_neighbors=k)
    knn_scores.append(cross_val_score(neigh_model, X=x, y=y, cv=10, scoring="accuracy").mean())

print(dict(zip(range(1, max_k + 1), knn_scores)))


{1: 0.7935770750988143, 2: 0.7312252964426877, 3: 0.8303359683794467, 4: 0.8113636363636365, 5: 0.8419960474308301, 6: 0.8328063241106719, 7: 0.8464426877470356, 8: 0.8424901185770752, 9: 0.8477272727272727, 10: 0.8460474308300394, 11: 0.8484189723320158, 12: 0.8464426877470356, 13: 0.8484189723320158, 14: 0.8471343873517787, 15: 0.8480237154150198, 16: 0.8461462450592885, 17: 0.8467391304347827, 18: 0.8463438735177865, 19: 0.8457509881422924, 20: 0.8464426877470356, 21: 0.8466403162055336, 22: 0.8465415019762844, 23: 0.8443675889328063, 24: 0.8452569169960474, 25: 0.8446640316205534, 26: 0.8457509881422924, 27: 0.8441699604743084, 28: 0.8441699604743083, 29: 0.8434782608695652, 30: 0.842885375494071}


In [60]:
best_k = max(zip(range(1, max_k + 1), knn_scores), key=lambda x: x[1])[0]
best_k

11

GridSearchCV

In [79]:
from sklearn.model_selection import GridSearchCV
k_range = range(1, max_k + 1)
weight_options = ["uniform", "distance"]
param_grid = dict(n_neighbors=k_range, weights=weight_options)
grid_search = GridSearchCV(estimator=neigh_model, param_grid=param_grid,
                    cv=10, scoring="accuracy")
grid_search.fit(X=x, y=y)
best_params = grid_search.best_params_


In [92]:
# neigh_model = KNeighborsClassifier(**best_params)
# cross_val_score(neigh_model, x, y, cv=10, scoring="accuracy").mean()
best_params

{'n_neighbors': 13, 'weights': 'distance'}

RandomizedSearchCV

In [94]:
from sklearn.model_selection import RandomizedSearchCV

rand_search = RandomizedSearchCV(estimator=neigh_model, param_distributions=param_grid,
                          n_iter=5, scoring="accuracy", cv=10)

rand_best_params_list = []
for i in range(10):
    rand_search.fit(X=x, y=y)
    rand_best_params_list.append((rand_search.best_score_, rand_search.best_params_))

best_params = max(rand_best_params_list, key=lambda x: x[0])[1]["n_neighbors"]

[(0.8487154150197629, {'weights': 'distance', 'n_neighbors': 17}),
 (0.849802371541502, {'weights': 'distance', 'n_neighbors': 13}),
 (0.849802371541502, {'weights': 'distance', 'n_neighbors': 13}),
 (0.8467391304347827, {'weights': 'uniform', 'n_neighbors': 17}),
 (0.849802371541502, {'weights': 'distance', 'n_neighbors': 13}),
 (0.8484189723320158, {'weights': 'distance', 'n_neighbors': 18}),
 (0.8463438735177865, {'weights': 'uniform', 'n_neighbors': 18}),
 (0.8485177865612649, {'weights': 'distance', 'n_neighbors': 21}),
 (0.8485177865612649, {'weights': 'distance', 'n_neighbors': 21}),
 (0.8457509881422924, {'weights': 'uniform', 'n_neighbors': 26})]